<a href="https://colab.research.google.com/github/periclesrialto/IndustrialAutomation/blob/master/AlarmMatrix_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
import math
import sys

In [70]:
#Script configs
#(xlsx or csv)
file_format = 'xlsx'
#(word or dword)
var_type = 'dword'
#################################################################
if var_type == 'dword':
  var_base_len = 32
elif var_type == 'word':
  var_base_len = 16
else:
  print('Invalid Type')
#################################################################
if file_format == 'xlsx':
  #reads .xlsx format
  df = pd.read_excel('Alarm_matrix.xlsx', header = None)
elif file_format == 'csv':
  #reads .csv format
  df = pd.read_csv('Alarm_matrix.csv', header = None, sep=';')
else:
  print('Invalid Format')

In [71]:
#Gather all the alarm categories on the table
alarm_category = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    shift = 1
    while df[column + shift][1] != 'ALL':
      alarm_category.append(df[column+ shift][1])
      shift += 1
    break
for category in range(len(alarm_category)):
  alarm_category[category] = alarm_category[category].lower()
  alarm_category[category] = alarm_category[category].capitalize()
  alarm_category[category] += 'Group'

#Gather all the machine states on the table
machine_state = []
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    machine_state.append(df[column][0])
for state in range(len(machine_state)):
  machine_state[state] = machine_state[state].replace(" ", "")

#Combines states and categories to create names for dataframe columns
column_names = []
for state in range(len(machine_state)):
  for category in range(len(alarm_category)):
    column_names.append(machine_state[state] + '.' + alarm_category[category])

#Create an empty dataframe using the previous generated names for the columns
#Insert the right amount of empty rows on the dataframe
dwords_df = pd.DataFrame(columns=column_names)
for empty in range(math.ceil(len(df.index)/var_base_len)):
  dwords_df = dwords_df.append(pd.Series(dtype='object'), ignore_index=True)
#dwords_df

In [72]:
#Creates an empty list of strings for
dwords_range = (math.ceil(len(df.index)/var_base_len))*len(alarm_category)*len(machine_state)
dwords_list = ["" for x in range(dwords_range)]

#Extracts all the bits from the imported table and combine them
pointer = 0
row = 0
for column in range(len(df.columns)):
  if df[column][1] == 'ALL':
    for shift in range(1,4):
      for index in range(2,len(df.index)):
        dwords_list[pointer] += str(df[column+shift][index])
        row += 1
        if row==var_base_len and pointer < len(dwords_list):
          pointer += 1
          row = 0
        if index == (len(df.index)-1):
          for complete in range(var_base_len-row):
            dwords_list[pointer] += str(0)
          if pointer < len(dwords_list):
            pointer += 1
            row = 0
for invert in range(len(dwords_list)):
  dwords_list[invert] += '#2'
  dwords_list[invert] = dwords_list[invert][::-1]
#dwords_list

In [73]:
pointer = 0
for column in range(len(dwords_df.columns)):
  for index in range(len(dwords_df.index)):
    dwords_df.at[index,column_names[column]]=dwords_list[pointer]
    pointer += 1
#dwords_df

In [97]:
#Combines all the previous data and create the code used on the PLC

print('(*')
print('!!!AVISO!!!')
print('Este código foi gerado automaticamente')
print('Para que o script em SCL funcione:\n')

print('Crie um array de alarmes:')
print(' --> ALM_AREA como Array[0..'+str(len(dwords_df.index)-1)+'] of '+var_type+'\n')

print('Crie um datatype que contenha:')
for category in range(len(alarm_category)):
  print(' --> '+str(alarm_category[category])+' como '+var_type)
  print(' --> '+str(alarm_category[category]).replace('Group','Out')+' como Bool')

print('\nCrie instâncias do datatype com os nomes:')
for state in range(len(machine_state)):
    print(' --> '+str(machine_state[state]))

print('\nCopie o script abaixo para o TIA Portal:\n')
print('*)\n\n')

#Finds the alarm text
print('REGION Alarms')
for column in range(len(df.columns)):
  if df[column][1] == 'Alarm (ctrl+c ctrl+v)':
    shift = 2
    counter = 0
    dword = 0
    for index in range(shift,len(df.index)):
      #Extract Alarm object
      sufix = ''
      alarm_str = 'AM' + str(df[column - 3][index]) + ' - '
      obj = ((str(df[column][index]).replace(alarm_str,'')).split(' ', 1)[0]).replace('-','_')
      if any(string in str(df[column][index]) for string in ['S43','Q10']):
        data_base = 'DB_BOTOES.'
      elif any(string in str(df[column][index]) for string in ['ZSH','ZSL']):
        data_base = 'DB_FC.'
      elif any(string in str(df[column][index]) for string in ['F9','F10', 'F11']):
        data_base = 'DB_PROTECAO.'
      elif any(string in str(df[column][index]) for string in ['SSL','PSL', 'LSH', 'LSL', 'HSV', 'PSH', 'PI', 'K59']):
        data_base = 'DB_SENSORES.'
      elif any(string in str(df[column][index]) for string in ['TT', 'ALTA']):
        data_base = 'DB_'
        sufix = '.HmiData.Op_Lim_A_HH'
      elif any(string in str(df[column][index]) for string in ['TT','BAIXA']):
        data_base = 'DB_'
        sufix = '.HmiData.Op_Lim_A_LL'
      elif any(string in str(df[column][index]) for string in ['FV','ABRIU']):
        data_base = 'DB_'
        sufix = '.Q_FALHA AND ' + data_base + obj + '.STW_VAL == 30'
      elif any(string in str(df[column][index]) for string in ['FV','FECHOU']):
        data_base = 'DB_'
        sufix = '.Q_FALHA AND ' + data_base + obj + '.STW_VAL == 31'
      elif any(string in str(df[column][index]) for string in ['FV','CONFLITO']):
        data_base = 'DB_'
        sufix = '.Q_FALHA AND ' + data_base + obj + '.STW_VAL == 32'
      else:
        data_base = ''
        sufix = ''

      print(' REGION ' + str(df[column][index]))
      if (obj != 'XYZ'):
        if (obj  != 'SYS'):
          print('   IF ' + data_base + obj + sufix + ' THEN ')
        elif (obj == 'SYS'):
          print('   IF Insira Condição de Sistema THEN ')
        print('     ALM_AREA[' + str(dword) + '].%X' + str(counter) + ' := TRUE;')
        print('   ELSIF ls_bReset THEN')
        print('     ALM_AREA[' + str(dword) + '].%X' + str(counter) + ' := FALSE;')
        print('   END_IF;')
        print(' END_REGION')
      else:
        print(' (* Alarme reserva')
        print('   IF ' + obj + ' THEN')
        print('     ALM_AREA[' + str(dword) + '].%X' + str(counter) + ' := TRUE;')
        print('   ELSIF ls_bReset THEN')
        print('     ALM_AREA[' + str(dword) + '].%X' + str(counter) + ' := FALSE;')
        print('   END_IF;')
        print(' *)')
        print(' END_REGION')
      counter += 1
      if counter == 32:
        counter = 0
        dword += 1
print('END_REGION\n')

print('REGION Alarms Masks \n')
category_pointer = 0
for column in range(len(dwords_df.columns)): 
  if category_pointer == 0:
    print(' REGION Masks '+str(dwords_df.columns[column].split('.')[0])+'\n')
  print('   REGION Comparing Alarm Matrix with '+str(dwords_df.columns[column])+'\n')
  for alarm_dw in range(len(dwords_df.index)):
    print('   IF (ALM_AREA['+str(alarm_dw)+'] AND '+str(dwords_df[dwords_df.columns[column]][alarm_dw])+') <> 0 THEN')
    print('     '+str(dwords_df.columns[column])+'.%X'+str(alarm_dw)+' := TRUE;')
    print('   ELSE')
    print('     '+str(dwords_df.columns[column])+'.%X'+str(alarm_dw)+' := FALSE;')
    print('   END_IF;\n')
    category_pointer += 1
  print('   REGION Compiles the '+str(" ".join(dwords_df.columns[column].split('.')))+' mask output\n')
  print('     IF '+str(dwords_df.columns[column])+' <> 0 THEN')
  print('       '+str(dwords_df.columns[column]).replace('Group','Out')+' := TRUE;')
  print('     ELSE')
  print('       '+str(dwords_df.columns[column]).replace('Group','Out')+' := FALSE;')
  print('     END_IF;\n')
  print('   END_REGION\n')
  print('   END_REGION\n')
  if category_pointer == (len(dwords_df.index)*len(alarm_category)):
    print(' END_REGION\n')
    category_pointer = 0
print('END_REGION\n\n')

(*
!!!AVISO!!!
Este código foi gerado automaticamente
Para que o script em SCL funcione:

Crie um array de alarmes:
 --> ALM_AREA como Array[0..6] of dword

Crie um datatype que contenha:
 --> EmergencyGroup como dword
 --> EmergencyOut como Bool
 --> EndGroup como dword
 --> EndOut como Bool
 --> InterruptGroup como dword
 --> InterruptOut como Bool

Crie instâncias do datatype com os nomes:
 --> Startup
 --> DryingUnder180
 --> DryingOver180
 --> Roasting
 --> Cooling
 --> Transport

Copie o script abaixo para o TIA Portal:

*)


REGION Alarms
 REGION AM001 - F9-2C - DISJUNTOR VENTILADOR AR QUENTE TORRADOR
   IF DB_PROTECAO.F9_2C THEN 
     ALM_AREA[0].%X0 := TRUE;
   ELSIF ls_bReset THEN
     ALM_AREA[0].%X0 := FALSE;
   END_IF;
 END_REGION
 REGION AM002 - F10-2C - DISJUNTOR CILINDRO TORRADOR
   IF DB_PROTECAO.F10_2C THEN 
     ALM_AREA[0].%X1 := TRUE;
   ELSIF ls_bReset THEN
     ALM_AREA[0].%X1 := FALSE;
   END_IF;
 END_REGION
 REGION AM003 - F9-4C - DISJUNTOR VENTILADOR ESFRIADOR